In [10]:
!pip install -q google-generativeai

In [11]:
import google.generativeai as genai
from google.colab import files

In [12]:
genai.configure(api_key="API KEY") #give your api key here
model = genai.GenerativeModel("models/gemini-1.5-flash")

In [13]:
session_history = []

In [14]:
def explain_code(code_snippet, language, mode, compare_code=None):
    if compare_code:
        prompt = (
            f"You are an AI code reviewer.\n"
            f"Compare the following two {language} code snippets.\n"
            f"Explain which one is better in terms of time and space complexity, and why.\n"
            f"Suggest improvements if possible.\n\n"
            f"First code:\n```{language.lower()}\n{code_snippet}\n```\n"
            f"Second code:\n```{language.lower()}\n{compare_code}\n```"
        )
    else:
        prompt = (
            f"You are an AI assistant that explains {language} code to beginners.\n"
            f"Explain the following {language} code clearly in simple English, "
            f"{'line by line' if mode == 'line-by-line' else 'as a summary'}.\n"
            f"Estimate time complexity and space complexity.\n"
            f"Point out any potential issues or bad practices.\n"
            f"Suggest an optimized version if possible.\n\n"
            f"```{language.lower()}\n{code_snippet}\n```"
        )
    response = model.generate_content(prompt)
    return response.text

In [15]:
def c(text, color):
    codes = {
        "pink": "\033[1;35m",
        "cyan": "\033[1;36m",
        "green": "\033[1;32m",
        "blue": "\033[1;34m",
        "red": "\033[1;31m",
        "end": "\033[0m"
    }
    return f"{codes.get(color, '')}{text}{codes['end']}"

In [16]:
session_count = 0
saved_files = []

In [18]:
while True:

    print(c("\n ========= AI CODE EXPLAINER =========", "pink"))
    print(c("1️ Enter your code", "cyan"))
    print(c("2️ Choose language", "cyan"))
    print(c("3️ Choose explanation type", "cyan"))
    print(c("4️ Generate explanation + analysis + suggestions", "cyan"))
    print(c("5 Save + Download explanation", "cyan"))
    print(c("======================================", "pink"))

    print(c("\n Please enter your main code (type END on a new line to finish):", "green"))
    code_lines = []
    while True:
        line = input()
        if line.strip().upper() == "END":
            break
        code_lines.append(line)
    code_input = "\n".join(code_lines)

    compare = input(c("Do you want to input another code to compare? (yes/no): ", "blue")).strip().lower()
    compare_code_input = None
    if compare == "yes":
        print(c(" Please enter the second code (type END on a new line to finish):", "green"))
        code_lines = []
        while True:
            line = input()
            if line.strip().upper() == "END":
                break
            code_lines.append(line)
        compare_code_input = "\n".join(code_lines)

    language = input(c("\nEnter programming language (default Python): ", "blue")).strip() or "Python"

    mode = input(c("Choose explanation type (line-by-line / summary, default line-by-line): ", "blue")).strip() or "line-by-line"

    print(c("\n🔹 Generating explanation, analysis, and suggestions...\n", "green"))
    explanation = explain_code(code_input, language, mode, compare_code=compare_code_input)
    print(c("===== AI EXPLANATION =====", "pink"))
    print(explanation)

    session_count += 1
    save_format = input(c("\nSave as txt or md? (default txt): ", "blue")).strip() or "txt"
    filename = f"ai_code_explanation_{session_count}.{save_format}"
    with open(filename, "w") as f:
        f.write(explanation)
    saved_files.append(filename)
    print(c(f"✅ Explanation saved to {filename}", "green"))
    files.download(filename)

    session_history.append({
        "code": code_input,
        "compare_code": compare_code_input,
        "language": language,
        "mode": mode,
        "file": filename
    })

    cont = input(c("\nDo you want to explain another code snippet? (yes/no): ", "blue")).strip().lower()
    if cont != "yes":
        break


 ========= AI CODE EXPLAINER =========
1️ Enter your code
2️ Choose language
3️ Choose explanation type
4️ Generate explanation + analysis + suggestions
5 Save + Download explanation

 Please enter your main code (type END on a new line to finish):
int linearSearch(int arr[], int n, int x) {     for (int i = 0; i < n; i++) {         if (arr[i] == x)             return i;     }     return -1; } END
END
Do you want to input another code to compare? (yes/no): yes
 Please enter the second code (type END on a new line to finish):
int binarySearch(int arr[], int left, int right, int x) {     if (right >= left) {         int mid = left + (right - left) / 2;         if (arr[mid] == x)             return mid;         if (arr[mid] > x)             return binarySearch(arr, left, mid - 1, x);         return binarySearch(arr, mid + 1, right, x);     }     return -1; } END
END

Enter programming language (default Python): c
Choose explanation type (line-by-line / summary, default line-by-line): sum

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Do you want to explain another code snippet? (yes/no): no


In [19]:
print(c("\nSESSION SUMMARY", "pink"))
print(f"- {session_count} code snippets explained")
print(f"- Files saved: {', '.join(saved_files)}")


SESSION SUMMARY
- 1 code snippets explained
- Files saved: ai_code_explanation_1.txt
